In [ ]:
import sys
import os
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
import pandas as pd
import numpy as np
import pickle

from sktime.utils.plotting import plot_series
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon

from sktime.forecasting.all import (
    MultiplexForecaster,
    NaiveForecaster,
    TrendForecaster,
    PolynomialTrendForecaster,
    ExponentialSmoothing,
    AutoETS,
    AutoARIMA,
    ThetaForecaster,
    Prophet,
    ForecastingGridSearchCV,
    ExpandingWindowSplitter,)

from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

In [ ]:
INITIAL_WINDOW_SIZE = 25
SP = 7
TEST_RATIO = 0.05

In [ ]:
# データをロード
with open('./datasets/7974.T_X.pkl', 'rb') as f:
    X = pickle.load(f)
    p_X = X.copy()
with open('./datasets/7974.T_y.pkl', 'rb') as f:
    y = pickle.load(f)
    p_y = y.copy()

# prophet用にindex変換
p_X.index = p_X.index.to_timestamp()
p_y.index = p_y.index.to_timestamp()

# targetの5%をtestデータに分割
y_train, y_test, X_train, X_test = temporal_train_test_split(
    y, 
    X, 
    test_size=int( len(y)*TEST_RATIO )
)
p_y_train, p_y_test, p_X_train, p_X_test = temporal_train_test_split(
    p_y, 
    p_X, 
    test_size=int( len(p_y)*TEST_RATIO )
)

len(y_train)

In [ ]:
forecaster = MultiplexForecaster(
    forecasters=[
    ("native", NaiveForecaster()),
    ("trend", TrendForecaster()),
    ("poly", PolynomialTrendForecaster()),
    ("exponent", ExponentialSmoothing()),
    ("ets", AutoETS(sp=SP)),
    ("arima", AutoARIMA(sp=SP)),
    ("theta", ThetaForecaster(sp=SP)),
    ("prophet", Prophet(seasonality_mode='multiplicative', add_country_holidays={'country_name': 'JP'}, daily_seasonality=True)),
    ])

cv = ExpandingWindowSplitter(
    fh=5,
    initial_window=INITIAL_WINDOW_SIZE,
    step_length=1,
    start_with_window=True
)
gscv = ForecastingGridSearchCV(
    cv=cv,
    param_grid={"selected_forecaster":[
        # "native",
        # "trend", 
        # "poly",  
        "exponent",
        "ets",
        #"arima",
        "theta",
        #"prophet" # DateTimeIndexである必要があるため別途トレーニング
        ]},
    forecaster=forecaster)
gscv.fit(y_train, X_train)

In [ ]:
gscv.best_forecaster_

In [ ]:

prophet_forecaster = Prophet(
        seasonality_mode='multiplicative',
        add_country_holidays={'country_name': 'JP'},
        daily_seasonality=True
    )
prophet_forecaster.fit(y=p_y_train, X=p_X_train)

In [ ]:
# forecasting horizon
fh = ForecastingHorizon(
    y_test.index, is_relative=False
)
prophet_fh = ForecastingHorizon(
    p_y_test.index, is_relative=False
)

In [ ]:
# 予測
y_pred = gscv.predict(fh=fh,X=X_test)
prophet_y_pred = prophet_forecaster.predict(fh=prophet_fh, X=p_X_test)
# 予測結果のグラフ化
prophet_y_pred.index = pd.PeriodIndex(prophet_y_pred.index, freq='D')
plot_series(y_train.tail(100), y_test, y_pred, prophet_y_pred, labels=["y_train", "y_test", "y_pred", "prophet_y_pred"])